## 二次判别分析Quadratic Discriminant Analysis(QDA)

二次判别分析是另外一种线性判别分析算法，它与LDA拥有类似的算法特征，区别仅在于：当不同分类样本的协方差矩阵相同时，使用线性判别分析；当不同分类样本的协方差矩阵不同时，则应该使用二次判别。

### QDA vs LDA

下图显示了在固定协方差矩阵以及不同协方差矩阵下LDA和QDA的表现差异：由图中可以看出，在固定协方差矩阵下，LDA和QDA是没有分类结果差异的（上面两张图）；但在不同的协方差矩阵下，LDA和QDA的分类边界明显存在差异，而且LDA已经不能准确的划分数据（下面两张图）。
![Text](plot_lda_qda.png)

### 协方差矩阵是什么？

协方差反映的是任意两个随机变量（或者是任意两个维度） 间的关系：
* 当X取值不断增大时，Y也不断增大，此时Cov(X,Y)>0
* 当X取值不断减小时，Y也不断减小，此时Cov(X,Y)>0
* 当X取值不断增大时，Y也不断减小，此时Cov(X,Y)<0
* 当X取值不断减小时，Y也不断增大，此时Cov(X,Y)<0

当两个随机变量倾向于沿着相同趋势变化时为正协方差，反之则为负协方差。
进一步就有了相关系数和协方差矩阵。